# Measurements in systems with multiple qubits

In the previous tutorial, we discussed the concept of measurements done on single qubit systems. Building upon those ideas, this tutorial will introduce you to the concepts involved in, and the `Q#` implementation of measurements done on multi-qubit systems. This will involved measuring a single qubit in a multi-qubit system, as well as measuring multiple qubits simultaneously. 

We recommend to go through the [tutorial that introduces single qubit system measurements](../SingleQubitGates/SingleQubitGates.ipynb) before starting this one.

This tutorial covers the following topics for multiple qubit systems:

* Measuring qubits one at a time
* Measuring multiple qubits simultaneously
$\newcommand{\ket}[1]{\left|#1\right>}$
$\newcommand{\bra}[1]{\left<#1\right|}$

## Types of measurements
Measurements in an n-qubit system ($n>1$) are of the following types:
- Measuring all the qubits simultaneously in an orthogonal basis ($2^n$ possible outcomes) .
- Pauli Measurements ($2$ possible outcomes).
- Partial Measurements ($2^m$ possible outcomes, if $m<n$ number of qubits are measured).

The first two types of measurements involve measuring all the qubits in a system but result in a different number of outcomes, while the last type involves measuring only some of the qubits in the system. Partial measurements are more subtle than complete measurements, and hence we discuss these topics in the same order as the list above. 


## Measurements in multi-qubit bases
Consider a system consisting of $n\geq1$ qubits. The wavefunction of such a system belongs to a vector space of dimension $2^n$. Thus, the vector space is spanned by an orthogonal basis such as the computational basis which consists of the vectors $|0\dotsc0\rangle, \dotsc, |1\dotsc 1\rangle$. For generality, we consider an arbitrary orthonormal basis, which we denote by $\{ |b_0\rangle, |b_1\rangle, \dotsc, |b_{2^n-1}\rangle \}$. Then, the state $|\psi\rangle$ of the multi-qubit system can be expressed as a linear combination of the $2^n$ basis vectors. That is, there exist complex numbers $c_0,c_1,\dotsc, c_{2^n-1}$ such that
 $$
 |\psi\rangle = \sum_{i=0}^{2^n-1} c_i|b_i\rangle \equiv \begin{pmatrix}c_0 \\ c_1 \\ \vdots \\ c_{2^n-1}\end{pmatrix}.
 $$
 In line with the usual convention, we choose the wavefunction to be normalized, so that $|c_0|^2 + \dotsc + |c_{2^n-1}|^2 =1$. Then, a quantum measurement in the $\{ |b_0\rangle, |b_1\rangle, \dotsc, |b_{2^n-1}\rangle \}$ basis satisfies the following rules:
 - The measurement outcome $b_i$ occurs with probability $|c_i|^2$. 
 - Whenever the measurement outcome is $b_i$, the wavefunction collapses to the state $|b_i\rangle$. That is, the post measurement state of the system is equal to $|b_i\rangle$.

This can be summarized in the following table:
<table style="border:1px solid">
    <col width=150>
    <col width=150>
    <col width=150>
    <tr>
        <th style="text-align:center; border:1px solid">Measurement outcome</th>
        <th style="text-align:center; border:1px solid">Probability of outcome</th>
        <th style="text-align:center; border:1px solid">State after measurement</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">$b_i$</td>
        <td style="text-align:center; border:1px solid">$|c_i|^2$</td>
        <td style="text-align:center; border:1px solid">$\ket{b_i}$</td>
    </tr>    
</table>
 
 
> Similar to measurements in single qubit systems, the assumption of normalization of the original wavefunction is required for the applicability of the statements above, in order to ensure that the sum of all the outcome probabilities is one.

### <span style="color:blue">Exercise 1</span>: Multi-qubit measurement outcome probabilities I 
Suppose that a two-qubit system is known to be in the following state:
$$\ket \psi =  \frac{1}{3}\ket {00} + \frac{2}{3} \ket {01} + \frac{2}{3}\ket {11}$$.

If all the qubits are measured simultaneously in the computational basis, what are the outcome probabilities?

*Can't come up with a solution? See the explained solution in the [--- Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-1:-Obtain-the-probability-outcomes-for-a-specific-state).*


### <span style="color:blue">Exercise 2</span>: Multi-qubit measurement outcome probabilities II 
Suppose that a two-qubit system is known to be in the following state:
$$\ket \psi =  \frac{2}{3}\ket {00} + \frac{1}{3} \ket {01} + \frac{2}{3}\ket {11}$$.

If all the qubits are measured simultaneously in the Pauli X basis, i.e. in the $\{ \ket{++}, \ket{+-}, \ket{-+}, \ket{--}\}$ basis, what are the outcome probabilities?

*Can't come up with a solution? See the explained solution in the [--- Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-1:-Obtain-the-probability-outcomes-for-a-specific-state).*


## Measuring all qubits in a system one after another
As described in the previous sections, in theory, it is possible to measure all the qubits in an n-qubit system simultaneously in an orthogonal basis. The post-measurement state of the qubits is then exactly one out of the $2^n$ possible basis states.

In practice, this is implemented by measuring all the qubits one after another. For example, if one wants to measure a two-qubit system in the computational basis, one can implement this by first measuring the first qubit in the computational basis to obtain 0 or 1, followed by a computational basis measurement for the second qubit. This can result in one of the four possible outcomes: $\ket{00}, \ket{01}, \ket{10}, \ket{11}$.

This can be generalized to measurements in other bases, such as $\ket{++}, \ket{+-}, \ket{-+}, \ket{--}$, and for a larger system sizes.


Although it is possible to measure all qubits in a quantum mechanics system simultaneously, in practice, the qubits are measured one after another on a quantum computer. 

> The distinction between measuring all the qubits simultaneously, as opposed to one after another is not important for an ideal computer: in terms of the outcomes and measurement probabilities, both are identical. Furthermore, as long as all the qubits are measured, the sequence in which they are measured is also inconsequential. (However, these factors can be  important to consider for real quantum computers with imperfect qubits, but we restrict the discussion here to ideal systems in this tutorial.) 

### <span style="color:blue">Demo: Basis state identification using qubit-by-qubit full measurement</span>
If a multi-qubit system is guaranteed to be in any one out of all the states in a specified orthogonal basis, then measuring the qubits in that basis can be used to identify the state of the qubits. In the cell below, we demonstrate this process for the computational basis. Specifically, we initialize a set of qubits in a randomly chosen computational basis state. Thereafter, we measure each qubit one after another in the the computational basis using the `M` operation in `Q#`. Finally, we print the measured state of the system. 

In [31]:
// Run this cell using Ctrl+Enter (⌘+Enter on Mac)
// Run the next cell to see the output

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Random;

operation IdentifyComputationalBasisDemo(): Unit {
    let NumQubits = 3;
    mutable ChosenBasisState = "";
    
    using (qs = Qubit[NumQubits]) {
        for (i in 0 .. NumQubits - 1) {
            // randomly choose True or False
            let RandomBool = DrawRandomBool (0.5);
            if (RandomBool) { 
                // if True, set qubit i to be in the |1❭ state
                // and update ChosenBasisState
                X(qs[i]);
                set ChosenBasisState = ChosenBasisState + "1";
            } else {
                set ChosenBasisState = ChosenBasisState + "0";
            }
        }
        
        Message($"Qubits initialized to the state state |{ChosenBasisState}❭ ");
    
        mutable MeasuredSystemState = "";
        mutable MeasuredQubitState = "";
        for (i in 0 .. Length(qs) - 1) {
            set MeasuredQubitState = M(qs[i])  == One ? "1" | "0";
            set MeasuredSystemState = MeasuredSystemState + MeasuredQubitState;
        }
        
        Message($"Qubits measured to be in the state |{MeasuredSystemState}❭");
        ResetAll(qs);
    }
}

IdentifyComputationalBasisDemo

In [47]:
%simulate IdentifyComputationalBasisDemo

Qubits initialized to the state state |010❭ 
Qubits measured to be in the state |010❭


()

> Running the cell above will result in a different output each time, but always with identical measured and initialized states, as expected.

### <span style="color:blue">Exercise 3</span>: Identify computational basis states

**Input**: An array of qubits which is guarangeed to be in a computational basis state.

**Output**: An array of `Result` values, with the same length as the number of qubits, specifying the computational basis state corresponding to the input state. For example, if the input qubit wavefunction were to be $\ket{001}$, the output should be the array `[Zero, Zero, One]`.

In [28]:
// %kata T3_ReturnBasisState

operation ReturnBasisState (qs : Qubit[]) : Result[] {
    
    let N = Length(qs);
    mutable array = new Result[N];
    
    // ... 
    
    // for (i in 0 .. N - 1) {
    //    set array w/= i <- M(qs[i]);
    // }
    // return array;
}

ReturnBasisState

## Pauli Measurements
 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Praesent tristique velit at lectus aliquam euismod. Nullam sed mauris arcu. Donec convallis nibh ac neque semper suscipit. Nam tristique id justo nec tincidunt. Vivamus ut tortor lacus. Vestibulum tempor laoreet condimentum. Donec blandit urna at pulvinar porttitor. Maecenas ac orci quis elit elementum tincidunt sit amet ut odio. Praesent egestas tortor quis lorem lacinia ultrices. Sed consequat purus nec semper placerat. 

## Partial Measurements
 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Praesent tristique velit at lectus aliquam euismod. Nullam sed mauris arcu. Donec convallis nibh ac neque semper suscipit. Nam tristique id justo nec tincidunt. Vivamus ut tortor lacus. Vestibulum tempor laoreet condimentum. Donec blandit urna at pulvinar porttitor. Maecenas ac orci quis elit elementum tincidunt sit amet ut odio. Praesent egestas tortor quis lorem lacinia ultrices. Sed consequat purus nec semper placerat. 

### <span style="color:blue">Exercise 4</span>: Distinguish orthogonal states

**Input**: A 3-qubit system which is guaranteed to be in either the $\ket{++-}$ state, or the $\ket{---}$ state.

**Output**: `true` if the qubit is in the $\ket{++-}$ state, or `false` if it was in the $\ket{---}$ state. The state of the qubits at the end of the operation should be the same as the initial state.

In [30]:
// %kata T4_DistinguishOrthogonalStates

operation DistinguishOrthogonalStates (qs : Qubit[]) : Bool {
    // ...
    return Measure([PauliX], [qs[0]]) == Zero;
}

DistinguishOrthogonalStates

First, we recall that a measurement in quantum systems is associated with some Hermitian operator. The different possible measurement outcomes correspond to the different eigenvalues of the Hermitian operator, while the post-measurement state is a projection onto the corresponding eigenspace. As an example, consider the computational basis measurements for a single qubit system. This corresponds to the operator $Z$ (i.e. the Pauli Z operator).


To add:
- Entanglement
- Partial measurements and projection operators